# Notebook Title: Database Insertion Functions for PostgreSQL
T
This notebook contains Python functions used to insert data into a PostgreSQL database. The functions are designed to automate the insertion of different entities (sites, formations, and modules) into respective database tables. The goal is to handle large folder structures, identify key elements, and populate the database efficiently.

The functions work together to:

    Normalize and format the data before inserting it.
    Check if the record already exists in the database before adding new entries.
    Handle errors and ensure that the database connection is managed properly.

Libraries:

The following Python libraries and modules are used in this notebook:

    psycopg2: A PostgreSQL adapter for Python to handle database connections and queries.
        Install it using: pip install psycopg2
    os: A standard Python library for interacting with the file system to traverse directories.
    connection: A custom module to establish a connection with the PostgreSQL database.
    util.common: A custom module for utility functions like normaliserModule and formaterModule used to standardize the module names before insertion.

Steps:
Step 1: insert_module

    Purpose: Inserts a new module into the modules table in the PostgreSQL database.
    Procedure:
        Normalize the module name: Standardize the format of the module name before inserting it.
        Database check: Queries the database to see if the module already exists.
        Insert if needed: Inserts the module if it doesn't already exist in the table.
        Error handling: Rolls back the transaction if an error occurs and closes the database connection.

Step 2: insert_formation

    Purpose: Inserts a new formation into the formations table in the PostgreSQL database.
    Procedure:
        Database connection: Establish a connection to the database.
        Check for formation: Queries the database to see if the formation already exists.
        Insert formation: If the formation doesn’t exist, it's inserted into the table.
        Commit changes: The transaction is committed to the database.
        Error handling: Handles errors and closes the database connection.

Step 3: insert_sites_formations_modules

    Purpose: Recursively traverses a folder structure to identify and insert sites, formations, and modules into the database.
    Procedure:
        Directory verification: Ensures the base folder exists and is a directory.
        Site traversal: Iterates through site folders and calls insert_formation to add formations.
        Formation traversal: For each site, it navigates into the formations, adding them to the database.
        Module insertion: Once the formations are added, the function dives deeper into each formation's subfolders to insert modules using insert_module.
        Completion: All relevant data from the folder structure is inserted into the database.


## 1. Function: insert_module


In [ ]:
def insert_module(module_name):
    """
    Inserts a new module into the 'modules' table of the database if it does not already exist.

    Parameters:
    module_name (str): The name of the module to be inserted.

    Returns:
    None
    """

    # Step 1: Normalize and format the module name
    module_name = normaliserModule(formaterModule(module_name))
    
    # Step 2: Establish a connection to the database
    conn = connection.get_connection()
    
    if conn is not None:
        try:
            # Step 3: Check if the module already exists
            with conn.cursor() as c:
                c.execute('SELECT id FROM modules WHERE name = %s', (module_name,))
                result = c.fetchone()
                
                # Step 4: If the module does not exist, insert it
                if result is None:
                    c.execute('INSERT INTO modules (name) VALUES (%s) RETURNING id', (module_name,))
                    new_id = c.fetchone()[0]
                    conn.commit()
                    print(f"Inserted module '{module_name}' with ID {new_id}")
                else:
                    print(f"Module '{module_name}' already exists with ID {result[0]}")
        
        # Step 5: Handle any errors and rollback in case of failure
        except (Exception, psycopg2.Error) as error:
            print(f"Error while inserting module: {error}")
            conn.rollback()
        
        # Step 6: Close the database connection
        finally:
            conn.close()


## 2. Function: insert_formation

In [ ]:
def insert_formation(formation_name):
    """
    Inserts a new formation into the 'formations' table of the database if it does not already exist.

    Parameters:
    formation_name (str): The name of the formation to be inserted.

    Returns:
    None
    """
    
    # Step 1: Establish a connection to the database
    conn = connection.get_connection()
    
    if conn is not None:
        try:
            # Step 2: Check if the formation already exists
            with conn.cursor() as c:
                c.execute('SELECT id FROM formations WHERE name = %s', (formation_name,))
                result = c.fetchone()
                
                # Step 3: If the formation does not exist, insert it
                if result is None:
                    c.execute('INSERT INTO formations (name) VALUES (%s) RETURNING id', (formation_name,))
                    new_id = c.fetchone()[0]
                    conn.commit()
                    print(f"Inserted formation '{formation_name}' with ID {new_id}")
                else:
                    print(f"Formation '{formation_name}' already exists with ID {result[0]}")
        
        # Step 4: Handle any errors and rollback in case of failure
        except (Exception, psycopg2.Error) as error:
            print(f"Error while inserting formation: {error}")
            conn.rollback()
        
        # Step 5: Close the database connection
        finally:
            conn.close()


## 3. Function: insert_sites_formations_modules

In [ ]:
def insert_sites_formations_modules(base_folder):
    """
    Recursively traverses the folder structure to insert sites, formations, and modules into the database.

    Parameters:
    base_folder (str): The path to the base folder containing sites, formations, and modules.

    Returns:
    None
    """
    
    # Step 1: Verify if the base folder exists and is a directory
    if os.path.exists(base_folder) and os.path.isdir(base_folder):
        
        # Step 2: Process each site folder within the base folder
        site_names = [name for name in os.listdir(base_folder) if os.path.isdir(os.path.join(base_folder, name))]
        for site_name in site_names:
            print("Checking site", site_name)
            site_folder = os.path.join(base_folder, site_name)
            
            # Step 3: Process each formation folder within the site folder
            formation_names = [name for name in os.listdir(site_folder) if os.path.isdir(os.path.join(site_folder, name))]
            for formation_name in formation_names:
                print("Checking formation", formation_name)
                insert_formation(formation_name)
                formation_folder = os.path.join(site_folder, formation_name)
                
                # Step 4: Process each module folder within the formation's "Modules" subfolder
                years_names = [name for name in os.listdir(formation_folder) if os.path.isdir(os.path.join(formation_folder, name))]
                for years_name in years_names:
                    print("Checking module", years_name)
                    years_folder = os.path.join(formation_folder, years_name) + "/Modules"
                    module_names = [name for name in os.listdir(years_folder) if os.path.isdir(os.path.join(years_folder, name))]
                    for module_name in module_names:
                        print("Checking module", module_name)
                        insert_module(module_name)

# Step 5: Specify the base folder path and call the function
sites_folder_path = "./Sites"
insert_sites_formations_modules(sites_folder_path)


Example of How to Run the Code:

    Run the Notebook: Make sure the database is properly set up, and the connection module (connection.py) points to the correct PostgreSQL instance.
    Directory Structure: Have your folder structure ready with Sites, Formations, and Modules subfolders to test the recursive insertion.
    Call the Functions: Start by calling insert_sites_formations_modules with the path to the base folder containing the directory structure.

Summary:

This notebook demonstrates the following key skills:

    Efficient data insertion into a PostgreSQL database.
    Folder traversal and handling dynamic structures.
    Error handling and transaction management.
    Modular and reusable code for database operations.